In [ ]:
# --- Robust SEC EDGAR quarterly chart (handles missing GrossProfit) ---

import time
from typing import Dict, List
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

# ---------- CONFIG ----------
UA = {"User-Agent": "POLITICO / Giulia Petrilli giuliapetrilli2000@gmail.com"}
BASE = "https://data.sec.gov/api"

# Tag candidates (in priority order)
REVENUE_TAGS = [
    "RevenueFromContractWithCustomerExcludingAssessedTax",  # ASC-606 "sales"
    "Revenues",
    "SalesRevenueNet"
]
GROSS_PROFIT_TAGS = ["GrossProfit"]  # may be absent
COST_TAGS = [
    "CostOfRevenue",
    "CostOfGoodsAndServicesSold"  # fallback name some filers use
]
NET_INCOME_TAGS = ["NetIncomeLoss"]

# ---------- UTIL / FETCH ----------
def load_ticker_map() -> Dict[str, str]:
    url = "https://www.sec.gov/files/company_tickers.json"
    r = requests.get(url, headers=UA, timeout=30)
    r.raise_for_status()
    j = r.json()
    return {v["ticker"].upper(): f'{int(v["cik_str"]):010d}' for v in j.values()}

def _get_json(url: str):
    r = requests.get(url, headers=UA, timeout=30)
    # handle 404 cleanly—means filer didn't use that tag
    if r.status_code == 404:
        return None
    r.raise_for_status()
    return r.json()

def company_concept(cik10: str, taxonomy: str, tag: str):
    url = f"{BASE}/xbrl/companyconcept/CIK{cik10}/{taxonomy}/{tag}.json"
    return _get_json(url)

def concept_to_df(j: dict, prefer_units=("USD", "USD$", "USD (in millions)")) -> pd.DataFrame:
    if j is None:
        return pd.DataFrame(columns=["fy","fp","end","val"])
    units = j.get("units", {})
    unit_key = None
    for u in prefer_units:
        if u in units:
            unit_key = u
            break
    if unit_key is None and units:
        unit_key = next(iter(units))
    rows = units.get(unit_key, [])
    if not rows:
        return pd.DataFrame(columns=["fy","fp","end","val"])
    df = pd.DataFrame(rows)
    for c in ("fy","fp","end","val"):
        if c not in df.columns: df[c] = pd.NA
    df["fy"] = pd.to_numeric(df["fy"], errors="coerce").astype("Int64")
    df["end"] = pd.to_datetime(df["end"], errors="coerce")
    return df.sort_values(["end","fy"], na_position="last").reset_index(drop=True)[["fy","fp","end","val"]]

def quarterly_series(df: pd.DataFrame) -> pd.DataFrame:
    q = df[df["fp"].str.upper().str.startswith("Q")].copy()
    if q.empty:
        return pd.DataFrame(columns=["fy","fp","end","value"])
    q["qnum"] = q["fp"].str.upper().str.extract(r"Q(\d)").astype("Int64")
    q = (q.sort_values("end")
           .groupby(["fy","qnum"], dropna=True)
           .agg(value=("val","last"), end=("end","last"))
           .reset_index())
    q["fp"] = "Q" + q["qnum"].astype("Int64").astype(str)
    return q[["fy","fp","end","value"]].sort_values(["fy","fp"])

def _fetch_first_available(cik10: str, tags: List[str], label: str = "") -> pd.DataFrame:
    """Try tags in order; return the first non-empty DF and print what was used."""
    for tag in tags:
        j = company_concept(cik10, "us-gaap", tag)
        if j is None:
            continue
        df = concept_to_df(j)
        if not df.empty:
            print(f"[{label}] Using tag: {tag}")
            return df
        time.sleep(0.2)
    print(f"[{label}] No data for tags: {tags}")
    return pd.DataFrame(columns=["fy","fp","end","val"])


def get_quarterly_financials(ticker: str, last_n_quarters: int = 16) -> pd.DataFrame:
    """
    Quarterly Revenue, Gross Profit (tag or derived), Net Income.
    """
    tmap = load_ticker_map()
    tk = ticker.upper()
    if tk not in tmap:
        raise ValueError(f"Ticker {ticker} not in SEC mapping.")
    cik10 = tmap[tk]

    # Revenue
    rev_df = _fetch_first_available(cik10, REVENUE_TAGS)
    rev_q = quarterly_series(rev_df).rename(columns={"value":"revenue"})

    time.sleep(0.2)

# Revenue

    # Gross Profit
    gp_df = _fetch_first_available(cik10, GROSS_PROFIT_TAGS, label="Gross Profit")
    gp_q  = quarterly_series(gp_df).rename(columns={"value":"gross_profit"})
    if gp_q.empty:
        cost_df = _fetch_first_available(cik10, COST_TAGS, label="Cost (for GP derivation)")
        cost_q  = quarterly_series(cost_df).rename(columns={"value":"cost"})
        gp_q = rev_q.merge(cost_q, on=["fy","fp","end"], how="left")
        gp_q["gross_profit"] = gp_q["revenue"] - gp_q["cost"]
        gp_q = gp_q[["fy","fp","end","gross_profit"]]
        print("[Gross Profit] Derived from Revenue – Cost")


    time.sleep(0.2)

    # Net Income
    ni_df = _fetch_first_available(cik10, NET_INCOME_TAGS)
    ni_q  = quarterly_series(ni_df).rename(columns={"value":"net_income"})

    # Combine
    df = rev_q.merge(gp_q, on=["fy","fp","end"], how="outer") \
              .merge(ni_q, on=["fy","fp","end"], how="outer") \
              .sort_values("end").reset_index(drop=True)

    df["gross_margin_pct"] = df["gross_profit"] / df["revenue"]
    df["net_margin_pct"]   = df["net_income"]  / df["revenue"]

    if last_n_quarters:
        df = df.tail(last_n_quarters).reset_index(drop=True)
    return df

# ---------- FORMATTING ----------
def _fmt_billions(x, pos):
    if np.isnan(x): return ""
    return f"${x/1e9:,.1f}B"

def _fmt_pct(x, pos):
    if np.isnan(x): return ""
    return f"{x*100:.1f}%"

# ---------- PLOTTING ----------
def plot_quarterly_performance(qdf: pd.DataFrame, company_name: str, ticker: str,
                               save_path: str = None):
    labels = [f"FY{str(int(fy))[-2:]}-{fp}" for fy, fp in zip(qdf["fy"], qdf["fp"])]
    x = np.arange(len(labels))
    width = 0.25

    fig, ax1 = plt.subplots(figsize=(12, 6))

    b1 = ax1.bar(x - width, qdf["revenue"], width, label="Revenue")
    b2 = ax1.bar(x,          qdf["gross_profit"], width, label="Gross Profit")
    b3 = ax1.bar(x + width,  qdf["net_income"], width, label="Net Income")

    ax1.set_ylabel("Billions USD")
    ax1.set_xlabel("Fiscal Quarter")
    ax1.set_xticks(x, labels, rotation=45, ha="right")
    ax1.yaxis.set_major_formatter(FuncFormatter(_fmt_billions))

    def _annotate(bars):
        for r in bars:
            h = r.get_height()
            if np.isnan(h): continue
            ax1.annotate(f"${h/1e9:,.1f}B",
                         xy=(r.get_x() + r.get_width()/2, h),
                         xytext=(0, 3),
                         textcoords="offset points",
                         ha="center", va="bottom", fontsize=7)
    _annotate(b1); _annotate(b2); _annotate(b3)

    ax2 = ax1.twinx()
    l1, = ax2.plot(x, qdf["gross_margin_pct"], marker="o", label="Gross Margin %")
    l2, = ax2.plot(x, qdf["net_margin_pct"], marker="o", label="Net Margin %")
    ax2.set_ylabel("Profit Margin (%)")
    ax2.yaxis.set_major_formatter(FuncFormatter(_fmt_pct))
    ycap = float(np.nanmax(qdf[["gross_margin_pct","net_margin_pct"]].values)) if not qdf.empty else 0.7
    ax2.set_ylim(0, max(0.5, ycap*1.2))

    for xi, yv in enumerate(qdf["gross_margin_pct"]):
        if pd.notna(yv):
            ax2.annotate(f"{yv*100:.1f}%", (x[xi], yv), textcoords="offset points",
                         xytext=(0, 6), ha="center", fontsize=7)
    for xi, yv in enumerate(qdf["net_margin_pct"]):
        if pd.notna(yv):
            ax2.annotate(f"{yv*100:.1f}%", (x[xi], yv), textcoords="offset points",
                         xytext=(0, -12), ha="center", fontsize=7)

    handles = [b1, b2, b3, l1, l2]
    labels = [h.get_label() for h in handles]
    ax1.legend(handles, labels, loc="upper left")

    fig.suptitle(f"{company_name} ({ticker.upper()}) — Quarterly Performance (Revenue, Profit & Margins)",
                 fontsize=14, fontweight="bold")
    fig.text(0.5, 0.02, "Source: SEC EDGAR (companyfacts JSON)", ha="center", fontsize=9)

    fig.tight_layout(rect=[0, 0.05, 1, 0.95])

    if save_path:
        fig.savefig(save_path, dpi=180, bbox_inches="tight")
    plt.show()

# ---------- EXAMPLE ----------
if __name__ == "__main__":
    TICKER = "QSR"                 # e.g., "MCD", "KO", "QSR"
    COMPANY_NAME = ""
    SAVE_TO = None                 # e.g., "mcd_quarterly.png"

    qdf = get_quarterly_financials(TICKER, last_n_quarters=16)
    print(qdf.tail())
    plot_quarterly_performance(qdf, COMPANY_NAME, TICKER, save_path=SAVE_TO)
